In [ ]:
import xml.sax
import sys
sys.path.append('/content/drive/My Drive/IRE')
import collections
import os
import os.path
import re
import re
from collections import defaultdict
import Stemmer
# from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
import timeit
from parsedata import Parser
import bz2
import glob
from heapq import heappush,heappop

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
!pip3 install PyStemmer
!pip3 install nltk

     |████████████████████████████████| 563kB 2.8MB/s 
  Created wheel for PyStemmer: filename=PyStemmer-2.0.1-cp36-cp36m-linux_x86_64.whl size=423413 sha256=81cee3cf66881e265c552c4f7eb4be15ef25ae73a58fa09aff59382102691d25
  Stored in directory: /root/.cache/pip/wheels/f3/3c/11/ee323a09706e17a649c2730bd8819b06e887411ff7507acf7a
Successfully built PyStemmer


In [ ]:
dump_folder=""
dump_files = list()
file_status = list()
file_count = 0
file_pointers = dict()
heap_words = list()
word_posting_list = dict()
global_dict = dict()
chunk_size = 170000
primary_file_count = 0
secondary_index = dict()

# get files from dump created by indexer(50000 doc in each)
def get_files_from_dump(dump_folder):
  global dump_files, file_status, file_count
  dump_files = glob.glob(dump_folder+'/indexfile*')
  # print(dump_files)
  # dump_files = dump_files[:6]
  dump_files.sort()
  # print(dump_files)
  file_count = len(dump_files)
  file_status = [0]*file_count

# from each index file pick up frist word and insert in heap if not present
def initialize_parameters():
  global file_status, file_pointers, word_posting_list
  for filenum in range(file_count):
    # print(dump_files[filenum])
    file_status[filenum] = 1
    file_pointers[filenum] = open(dump_files[filenum], 'r')
    word_posting_list[filenum] = file_pointers[filenum].readline().split(':')
    word = word_posting_list[filenum][0]
    # print(word)
    if word not in heap_words:
      # print('pushed', word)
      heappush(heap_words, word)

def push_word_in_heap(filenum, next_list):
  global heap_words, word_posting_list
  word_posting_list[filenum] = next_list.split(':')
  word = word_posting_list[filenum][0]
  if word not in heap_words:
    heappush(heap_words, word)

def update_status(filenum):
  global file_pointers, file_status
  file_pointers[filenum].close()
  file_status[filenum] = 0

# global dict contains chunksize words --- flush to primary file and write first word with primary file name to secondary index
def create_primary_index_file():
  global primary_file_count, global_dict
  primary_file_count += 1
  insert_in_sec = True
  primary_index_file = '/content/primaryfile_'+str(primary_file_count)+'.txt'
  
  with open(primary_index_file, "w") as of:
    for word in sorted(global_dict):
      if insert_in_sec:
        secondary_index[word] = 'primaryfile_' + str(primary_file_count)+'.txt'
        insert_in_sec = False
      # print(word + ':' + '|'.join(global_dict[word]))
      of.write(word + ':' + '|'.join(global_dict[word])+'\n')
  of.close()

def create_secondary_index_file():
  global secondary_index
  secondary_index_file = '/content/secondaryfile.txt'
  with open(secondary_index_file, "w") as of:
    for i in sorted(secondary_index):
      of.write(i + ":" + secondary_index[i] + '\n')
  of.close()

#k way merge -- pick first word from heap check for doc with first word and append the posting list for that word in global dict
def merge_index_files():
  global file_status, file_pointers, word_posting_list, global_dict, chunk_size
  currwords = 0
  while file_status.count(0) < file_count:

    topval = heappop(heap_words)
    currwords += 1
    global_dict[topval] = []
    for i in range(file_count):
      if file_status[i] == 1 and word_posting_list[i][0] == topval:
        posting_list = word_posting_list[i][1][:-1]
        # print("before ", posting_list)
        global_dict[topval].append(posting_list)
        # print("after ", global_dict[topval])
        next_list = file_pointers[i].readline()

        if next_list:
          push_word_in_heap(i, next_list)
        else:
          update_status(i)
    
    if currwords >= chunk_size:
      create_primary_index_file()
      currwords = 0
      global_dict.clear() 
      # return
      # sys.exit()




if (__name__ == "__main__"):

  dump_folder = "/content"
  get_files_from_dump(dump_folder)
  # print(file_count)
  initialize_parameters()
  merge_index_files()
  if bool(global_dict):
    create_primary_index_file()
  # flush secondary index to secondary file
  create_secondary_index_file()


